In [2]:
### Install necessary libraries
!pip install beautifulsoup4

In [111]:
### Install Snowflakes libraries
!pip install snowflake-sqlalchemy snowflake-connector-python

     ---------------------------------------- 0.0/66.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/66.7 kB ? eta -:--:--
     ----------------------- -------------- 41.0/66.7 kB 653.6 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/66.7 kB 653.6 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     ----------------------------- -------- 51.2/66.7 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 66.7/66.7 kB 133.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/43.7 kB

In [115]:
## Import the necessary libraries
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np
import re, os
from dotenv import load_dotenv

In [8]:
## Checking if website allows web scrapping by checking for url status
url = 'https://www.jumia.com.ng/'
result = rq.get(url)
print(result.status_code) # We're hoping for a status code of 200

200


##### Bring in BeautifulSoup

In [30]:
## Parse the response above as html
soup = BeautifulSoup(result.text, "lxml")
soup

<!DOCTYPE html>
<html dir="ltr" lang="en"><head><meta charset="utf-8"/><title>Samsung Galaxy S23 5G - 6.1", 128GB ROM,8GB RAM, Single Sim - Black | Jumia Nigeria</title><meta content="product" property="og:type"/><meta content="Jumia Nigeria" property="og:site_name"/><meta content='Galaxy S23 5G - 6.1", 128GB ROM,8GB RAM, Single Sim - Black' property="og:title"/><meta content="Brand / Model Samsung Galaxy S23 S911UOS Android 13CPU Snapdragon 8 Gen 2GPU Adreno 740RAM 8 GBROM 128 GB Display
 Size 6.1 inchScreen Resolution 2340 x 1080Screen Type Dynamic AMOLED 
2XLanguages Default English, Italian, Czech, German, Russian, 
Portuguese, Spanish, French, Dutch, Polish, Romanian, Danish, Greek, 
Brazil Portuguese, Croatian, Latvian, Lithuanian, Hungarian, Afrikaner, 
Ethiopic, Indonesian, Malay, Catalan, Norwegian, Roma, Slovak, 
Slovenian, Finnish, Swedish, Swahili, Filipino, Indonesian, Vietnamese, 
Turkish, Zulu, Bulgarian, Serbian, Ukrainian, Persian, Hindi, Thai, 
Korean, Japanese, Arabi

In [11]:
print(soup.body.div.main.div.div.div.a)

<a class="itm" href="https://www.jumia.com.ng/mlp-appliances/" role="menuitem"><svg class="ic -mrxs -fsh0" height="20" viewbox="0 0 24 24" width="20"><use xlink:href="https://www.jumia.com.ng/assets_he/images/i-icons.995b8ca3.svg#cat-kitchen-dinning"></use></svg><span class="text">Appliances</span></a>


##### Attributes

In [12]:
tag = soup.body.div.main.div.div.div.a
print(tag.attrs)

{'href': 'https://www.jumia.com.ng/mlp-appliances/', 'class': ['itm'], 'role': 'menuitem'}


In [14]:
tag_result = tag.attrs
print(tag_result['role'])

menuitem


##### Find functions - find(), find_all(), and regular expressions

In [38]:
## Find()

url = 'https://www.jumia.com.ng/catalog/?q=Camera'
result = rq.get(url)
soup = BeautifulSoup(result.text, "lxml")
print(soup)

<!DOCTYPE html>
<html dir="ltr" lang="en"><head><meta charset="utf-8"/><title>Shop All Products - TVs, Laptops, Fashion Items | Jumia Nigeria</title><meta content="product" property="og:type"/><meta content="Jumia Nigeria" property="og:site_name"/><meta content="Shop All Products - TVs, Laptops, Fashion Items | Jumia Nigeria" property="og:title"/><meta content="Enjoy amazing discounts and deals up to 70% on your favourite iPhones, android devices, TVs, Cookers and more offers at the best prices on Jumia Nigeria." property="og:description"/><meta content="/catalog/" property="og:url"/><meta content="https://ng.jumia.is/cms/jumialogonew.png" property="og:image"/><meta content="en_NG" property="og:locale"/><meta content="Shop All Products - TVs, Laptops, Fashion Items | Jumia Nigeria" name="title"/><meta content="noindex,follow" name="robots"/><meta content="Enjoy amazing discounts and deals up to 70% on your favourite iPhones, android devices, TVs, Cookers and more offers at the best pri

In [27]:
url = 'https://www.jumia.com.ng/samsung-galaxy-s23-5g-6.1-128gb-rom8gb-ram-single-sim-black-342555094.html'
result = rq.get(url)
soup = BeautifulSoup(result.text, "lxml")
print(soup.find('h1', {'class':"-fs20 -pts -pbxs"}))

<h1 class="-fs20 -pts -pbxs">Samsung Galaxy S23 5G - 6.1", 128GB ROM,8GB RAM, Single Sim - Black</h1>


##### Checking Camera Prices

In [ ]:
url = 'https://www.jumia.com.ng/catalog/?q=Camera'
result = rq.get(url)
soup = BeautifulSoup(result.text, "lxml")
print(soup)

In [42]:
### Extracting information - find(tag, attribute)
single_find = soup.find('div', class_ = 'info')
single_find

<div class="info"><h3 class="name">Andoer TFT Portable Digital Camera 48MP 4K Ultra HD 16X Zoom - 3in</h3><div class="prc">₦ 76,270</div><div class="s-prc-w"><div class="old">₦ 144,920</div><div class="bdg _dsct _sm">47%</div></div><div class="rev"><div class="stars _s">4 out of 5<div class="in" style="width:80%"></div></div>(11)</div><svg aria-label="Express Shipping" class="ic xprss" height="10" viewbox="0 0 114 12" width="94"><use xlink:href="https://www.jumia.com.ng/assets_he/images/i-shop-jumia.9f5451c7.svg#express"></use></svg></div>

In [44]:
## Nested Find
another_find = single_find.find('h3', class_ = 'name').text
another_find

'Andoer TFT Portable Digital Camera 48MP 4K Ultra HD 16X Zoom - 3in'

In [45]:
another_find = single_find.find('div', class_ = 'prc').text
another_find

'₦ 76,270'

In [48]:
another_find = single_find.find('div', class_ = 'stars _s').text
another_find

'4 out of 5'

In [40]:
### Extracting information - findall(tag, attribute)
soup.find_all('div', class_ = 'info')

[<div class="info"><h3 class="name">Andoer TFT Portable Digital Camera 48MP 4K Ultra HD 16X Zoom - 3in</h3><div class="prc">₦ 76,270</div><div class="s-prc-w"><div class="old">₦ 144,920</div><div class="bdg _dsct _sm">47%</div></div><div class="rev"><div class="stars _s">4 out of 5<div class="in" style="width:80%"></div></div>(11)</div><svg aria-label="Express Shipping" class="ic xprss" height="10" viewbox="0 0 114 12" width="94"><use xlink:href="https://www.jumia.com.ng/assets_he/images/i-shop-jumia.9f5451c7.svg#express"></use></svg></div>,
 <div class="info"><div class="bdg _glb _xs">Shipped from abroad</div><h3 class="name">1.3 Mega Pixel Interpolation Flip Screen  Interchangeable Lens Digital Camera, 2.4 Inch LCD, Full HD 720P Recording, Infrared Lens, EIS</h3><div class="prc">₦ 67,747</div><div class="s-prc-w"><div class="old">₦ 121,944</div><div class="bdg _dsct _sm">44%</div></div><div class="rev"><div class="stars _s">4.5 out of 5<div class="in" style="width:90%"></div></div>(2

##### Extracting required data

In [46]:
names = []
prices = []
discounts = []
old_prices = []
ratings = []

url = 'https://www.jumia.com.ng/catalog/?q=Camera'
result = rq.get(url)
soup = BeautifulSoup(result.text, "lxml")

In [49]:
### Extracting information - findall(tag, attribute)
raw_listings = soup.find_all('div', class_ = 'info')

## Getting all our details
for item in raw_listings:
    # Retrieve Camera names
    name = item.find('h3', class_ = 'name').text
    names.append(name)

    # Retrieve Camera prices
    price = item.find('div', class_ = 'prc').text
    prices.append(price)

    # Retrieve Camera old prices
    if item.find('div', class_ = 's-prc-w'):
        old_price = item.find('div', class_ = 'old').text
        old_prices.append(old_price)

        # Retrieve Camera discounts
        discount = item.find('div', class_ = 'bdg _dsct _sm').text
        discounts.append(discount)
    else:
        old_prices.append(np.nan)
        discounts.append(np.nan)

    # Retrieve Camera ratings
    if item.find('div', class_ = 'stars _s'):
        rating = item.find('div', class_ = 'stars _s').text
        ratings.append(rating)
    else:
        ratings.append(np.nan)

In [50]:
## Making the DataFrame
camera_data = pd.DataFrame({'name':names,
                            'price':prices,
                            'old_price': old_prices,
                            'discount': discounts,
                            'rating': ratings})

In [51]:
camera_data.head()

,name,price,old_price,discount,rating
0,Andoer TFT Portable Digital Camera 48MP 4K Ult...,"₦ 76,270","₦ 144,920",47%,4 out of 5
1,1.3 Mega Pixel Interpolation Flip Screen Inte...,"₦ 67,747","₦ 121,944",44%,4.5 out of 5
2,4K Night Vision HD WIFI Digital Camera Standar...,"₦ 110,285","₦ 157,549",30%,3.5 out of 5
3,KOMERY AF2 4K Digital Video Camera With Stabil...,"₦ 386,039","₦ 551,484",30%,5 out of 5
4,X6 Portable Ultra Mini High Denifition Digital...,"₦ 16,130","₦ 30,650",47%,NaN


In [52]:
camera_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       40 non-null     object
 1   price      40 non-null     object
 2   old_price  38 non-null     object
 3   discount   38 non-null     object
 4   rating     30 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [ ]:
### The above data is just for 1 page.
### Now lets run through all the pages

In [53]:
count = 1
url = 'https://www.jumia.com.ng/catalog/?q=camera&page={count}#catalog-listing'
response = rq.get(url)
response.status_code

200

In [71]:
run_it = True
count = 1

names = []
prices = []
discounts = []
old_prices = []
ratings = []

while run_it:

    url = f'https://www.jumia.com.ng/catalog/?q=camera&page={count}#catalog-listing'
    response = rq.get(url)

    if response.status_code != 200:
        print(f"There was an error with the call. The error code is {response.status_code}!")
        break

    soup = BeautifulSoup(response.text, 'lxml')
    
    check_msg = soup.find('h2', class_ = '-pvs -fs16 -m -ow-a')
    if check_msg and 'There are no results' in check_msg.text:
        print(f"We've run out of pages. Robot terminating tasks now...")
        break

    raw_listings = soup.find_all('div', class_ = 'info')

    ## Getting all our details
    for item in raw_listings:
        # Retrieve Camera names
        name = item.find('h3', class_ = 'name').text
        names.append(name)

        # Retrieve Camera prices
        price = item.find('div', class_ = 'prc').text
        prices.append(price)

        # Retrieve Camera old prices
        if item.find('div', class_ = 's-prc-w'):
            old_price = item.find('div', class_ = 'old').text
            old_prices.append(old_price)

            # Retrieve Camera discounts
            discount = item.find('div', class_ = 'bdg _dsct _sm').text
            discounts.append(discount)
        else:
            old_prices.append(np.nan)
            discounts.append(np.nan)

        # Retrieve Camera ratings
        if item.find('div', class_ = 'stars _s'):
            rating = item.find('div', class_ = 'stars _s').text
            ratings.append(rating)
        else:
            ratings.append(np.nan)
    
    print(f"Completed for run {count}")
    count+=1

Completed for run 1
Completed for run 2
Completed for run 3
Completed for run 4
Completed for run 5
Completed for run 6
Completed for run 7
Completed for run 8
Completed for run 9
Completed for run 10
Completed for run 11
Completed for run 12
Completed for run 13
Completed for run 14
Completed for run 15
Completed for run 16
Completed for run 17
Completed for run 18
Completed for run 19
Completed for run 20
Completed for run 21
Completed for run 22
Completed for run 23
Completed for run 24
Completed for run 25
Completed for run 26
Completed for run 27
Completed for run 28
Completed for run 29
Completed for run 30
Completed for run 31
Completed for run 32
Completed for run 33
Completed for run 34
Completed for run 35
Completed for run 36
Completed for run 37
Completed for run 38
Completed for run 39
Completed for run 40
Completed for run 41
Completed for run 42
Completed for run 43
Completed for run 44
Completed for run 45
Completed for run 46
Completed for run 47
Completed for run 48
C

In [72]:
## Making the DataFrame
all_camera_data = pd.DataFrame({'name':names,
                            'price':prices,
                            'old_price': old_prices,
                            'discount': discounts,
                            'rating': ratings})

In [73]:
all_camera_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       1999 non-null   object
 1   price      1999 non-null   object
 2   old_price  1651 non-null   object
 3   discount   1651 non-null   object
 4   rating     430 non-null    object
dtypes: object(5)
memory usage: 78.2+ KB


In [74]:
all_camera_data.head()

,name,price,old_price,discount,rating
0,Andoer TFT Portable Digital Camera 48MP 4K Ult...,"₦ 76,270","₦ 144,920",47%,4 out of 5
1,1.3 Mega Pixel Interpolation Flip Screen Inte...,"₦ 67,747","₦ 121,944",44%,4.5 out of 5
2,4K Night Vision HD WIFI Digital Camera Standar...,"₦ 110,285","₦ 157,549",30%,3.5 out of 5
3,KOMERY AF2 4K Digital Video Camera With Stabil...,"₦ 386,039","₦ 551,484",30%,5 out of 5
4,X6 Portable Ultra Mini High Denifition Digital...,"₦ 16,130","₦ 30,650",47%,NaN


In [76]:
trans_data = all_camera_data.copy()

##### Data Transformation

In [84]:
## Remove =N= sign and space from the amount fields
trans_data['price'] = trans_data['price'].apply(lambda x: x.split(" ")[1].replace(',', ''))

trans_data['old_price'] = trans_data['old_price'].apply(lambda x: x.split(" ")[1].replace(',', '') if isinstance(x, str) else np.nan)

In [92]:
## Remove the % sign from the discount field
trans_data['discount'] = trans_data['discount'].apply(lambda x: x.replace('%', '') if isinstance(x, str) else np.nan)

In [94]:
## Update our rating column to only have the first value
trans_data['rating'] = trans_data['rating'].apply(lambda x: x.split(' ')[0] if isinstance(x, str) else np.nan)

In [85]:
## Converting the datatypes
trans_data['price'] = trans_data['price'].astype(float)
trans_data['old_price'] = trans_data['old_price'].astype(float)
trans_data['discount'] = trans_data['discount'].astype(float)
trans_data['rating'] = trans_data['rating'].astype(float)

In [96]:
trans_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       1999 non-null   object 
 1   price      1999 non-null   float64
 2   old_price  1651 non-null   float64
 3   discount   1651 non-null   float64
 4   rating     430 non-null    float64
dtypes: float64(4), object(1)
memory usage: 78.2+ KB


In [98]:
trans_data.head()

,name,price,old_price,discount,rating
0,Andoer TFT Portable Digital Camera 48MP 4K Ult...,76270.0,144920.0,47.0,4.0
1,1.3 Mega Pixel Interpolation Flip Screen Inte...,67747.0,121944.0,44.0,4.5
2,4K Night Vision HD WIFI Digital Camera Standar...,110285.0,157549.0,30.0,3.5
3,KOMERY AF2 4K Digital Video Camera With Stabil...,386039.0,551484.0,30.0,5.0
4,X6 Portable Ultra Mini High Denifition Digital...,16130.0,30650.0,47.0,NaN


In [117]:
load_dotenv(override=True)
username = os.getenv('username')
password = os.getenv('password')
accnt_id = os.getenv('accnt_id')
database = os.getenv('database')
dbschema = os.getenv('dbschema')
warehuse = os.getenv('warehuse')

In [118]:
## Loading data to Snowflake

engine = create_engine(
    f'snowflake://{username}:{password}@{accnt_id}/{database}/{dbschema}?warehouse_name={warehuse}'
)

In [119]:
with engine.connect() as connection:
    connection.execute(text(f'USE SCHEMA {dbschema}'))

    trans_data.to_sql('src_Camera', con=engine, if_exists='append', index=False)